Unicode normalization is used to normalize different but similiar characters.  Two different types of equivalence here, canonical and compatibility equivalence.

* **Canonical equivalence** means both forms are fundamentally the same and when rendered are indistinguishable. For example we can take the unicode for `'Ç' \u00C7` or the unicode for `'C' \u0043` and `'̧' \u0327`, when the latter two characters are rendered together they look the same as the first character:

In [1]:
print("\u00C7", "\u0043"+"\u0327")

Ç Ç


However, if we print these characters seperately, we can see very clearly that they are not the same

In [2]:
print("\u00C7", "\u0043", "\u0327")

Ç C ̧


These are examples of canonical equivalence, but we also have compatibility equivalence.

**Compatibility equivalence** refers to the formatting differences between characters, which includes (but is not limited to):

* font variants
* cursive forms
* circled characters
* width variation
* size changes
* rotation
* superscript and subscript
* fractions

In this case we can see a difference between the rendered characters, for example between `ℌ` and `H`, or `½` and `1 ⁄ 2`.

For many of these examples which are either canonical equivalents (Ç ↔ C ̧ ) or compatibility equivalents (½ → 1 ⁄ 2), if we compare if these different forms are equal, we will find that they are not:

In [3]:
"Ç" == "Ç"

False

In [4]:
"ℌ" == "H"

False

So it is in these cases that we use unicode normalization to normalize our characters into matching pairs. As there are different forms of equivalence, there are also different forms of normalization. These are all called **N**ormal **F**orm, and there are four different methods:

* NFD: Canonical decomposition `Ç → C`
* NFC:Canoncial decomposition followed by canonical composition	`Ç → C ̧ → Ç`
* NFKD:	Compatibility decomposition	`ℌ ̧ → H` ̧
* NFKC:	Compatibility decomposition followed by canonical composition `ℌ ̧ → H ̧ → Ḩ`

In [5]:
c_with_cedilla = "\u00C7"  # Latin capital C with cedilla (single character)
c_with_cedilla

'Ç'

In [6]:
c_plus_cedilla = "\u0043\u0327"  # \u0043 = Latin capital C, \u0327 = 'combining cedilla' (two characters)
c_plus_cedilla

'Ç'

In [7]:
c_with_cedilla == c_plus_cedilla

False

If we perform NFD on our C with cedilla character \u00C7, we decompose the character into it's smaller components, which are the Latin capital C character, and combining cedilla character \u0043 + \u0327. This means that if we compare an NFD normalized C with cedilla character to both the C character and the cedilla character, we will return true

In [9]:
import unicodedata
unicodedata.normalize('NFD', c_with_cedilla) == c_plus_cedilla

True

However, if we perform NFC on our C with cedilla character \u00C7, we decompose the character into the smaller components \u0043 + \u0327, and then compose them back to \u00C7, and so they will not match

In [10]:
unicodedata.normalize('NFC', c_with_cedilla) == c_plus_cedilla

False

But if we switch the NFC encoding to instead be performed on our two characters \u0043 + \u0327, they will first be decomposed (which will do nothing as they are already decomposed), then compose them into the single \u00C7 characte

In [11]:
c_with_cedilla == unicodedata.normalize('NFC', c_plus_cedilla)

True

The NFK encodings do not decompose characters into smaller components, they decompose characters into their normal versions. For example if we take the fancy format ℌ \u210B, we cannot decompose this into multiple characters and so NFD or NFC encoding will do nothing. However, if we apply NFKD, we will find that our fancy ℌ \u210B becomes a plain, boring H \u0048

In [12]:
unicodedata.normalize('NFKD', 'ℌ')

'H'

But, what if we merge our fancy ℌ \u210B with a combining cedilla \u0328 character?

In [13]:
"\u210B\u0327"

'ℋ̧'

Applying our compatibility decomposition normalization (NFKD) gives us a capital H character, and a combining cedilla character as two seperate encodings:

In [14]:
unicodedata.normalize('NFKD', "\u210B\u0327").encode('utf-8')

b'H\xcc\xa7'

But if we apply NFKC, we first perform compatibility decomposition, into the two seperate characters, before merging them during canonical composition

In [15]:
unicodedata.normalize('NFKC', "\u210B\u0327").encode('utf-8')

b'\xe1\xb8\xa8'

Because the only difference between these two methods is a canonical composition, we see no difference between the two character sets when they are rendered

In [16]:
unicodedata.normalize('NFKC', "\u210B\u0327"), unicodedata.normalize('NFKD', "\u210B\u0327")

('Ḩ', 'Ḩ')